We first load the necessary packages:

In [510]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import OneHotEncoder, StandardScaler

from sklearn.linear_model import LinearRegression

We fetch the data:

In [511]:
data_path = os.getcwd()
data_file = 'VesselData.xlsx'
data = pd.read_excel(data_path + '/' + data_file)

Let's take a look at the data:

In [512]:
data.head()

,eta,ata,atd,vesseldwt,vesseltype,discharge1,load1,discharge2,load2,discharge3,...,load4,stevedorenames,hasnohamis,earliesteta,latesteta,traveltype,previousportid,nextportid,isremarkable,vesselid
0,2017-09-19 00:00:00+00,2017-09-19 00:00:00+00,2017-09-22 00:00:00+00,109290.0,5.0,0.0,0.0,0.0,0.0,90173.0,...,0.0,Stevedore_104,NaN,2017-09-19 00:00:00+00,2017-09-19 00:00:00+00,ARRIVAL,981.0,731.0,f,2242.0
1,2017-10-02 00:00:00+00,2017-10-02 00:00:00+00,2017-10-03 00:00:00+00,67170.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,Stevedore_109,NaN,2017-10-02 00:00:00+00,2017-10-02 00:00:00+00,ARRIVAL,19.0,15.0,f,5462.0
2,2017-09-30 00:00:00+00,2017-09-30 00:00:00+00,2017-10-01 00:00:00+00,67737.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,Stevedore_57,NaN,2017-09-30 00:00:00+00,2017-09-30 00:00:00+00,ARRIVAL,19.0,19.0,f,5251.0
3,2017-10-02 00:00:00+00,2017-10-02 00:00:00+00,2017-10-03 00:00:00+00,43600.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,Stevedore_57,NaN,2017-10-02 00:00:00+00,2017-10-02 00:00:00+00,ARRIVAL,15.0,18.0,f,5268.0
4,2017-10-02 00:00:00+00,2017-10-02 00:00:00+00,2017-10-02 00:00:00+00,9231.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,Stevedore_98,NaN,2017-10-02 00:00:00+00,2017-10-02 00:00:00+00,ARRIVAL,74.0,27.0,f,5504.0


In [513]:
print(f'The test data has {data.shape[0]} rows and {data.shape[1]} columns')

The test data has 8208 rows and 22 columns


In [514]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8208 entries, 0 to 8207
Data columns (total 22 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   eta             8208 non-null   object 
 1   ata             8208 non-null   object 
 2   atd             8208 non-null   object 
 3   vesseldwt       8206 non-null   float64
 4   vesseltype      8208 non-null   float64
 5   discharge1      8208 non-null   float64
 6   load1           8208 non-null   float64
 7   discharge2      8208 non-null   float64
 8   load2           8208 non-null   float64
 9   discharge3      8208 non-null   float64
 10  load3           8208 non-null   float64
 11  discharge4      8208 non-null   float64
 12  load4           8208 non-null   float64
 13  stevedorenames  8206 non-null   object 
 14  hasnohamis      0 non-null      float64
 15  earliesteta     8208 non-null   object 
 16  latesteta       8208 non-null   object 
 17  traveltype      8208 non-null   o

We now look for missing or duplicated data:

In [515]:
def check_missing(dataset):
    
    num_missing_percol = dataset.isnull().sum().sort_values(ascending=False)

    col_with_missing = num_missing_percol[num_missing_percol > 0]
    percentage = col_with_missing/dataset[col_with_missing.index].shape[0]

    df_missing = pd.concat([col_with_missing, round(percentage*100,1)], keys = ['Missing Values', 'Percentage %'], axis = 1)

    return df_missing 

In [516]:
check_missing(data)

,Missing Values,Percentage %
hasnohamis,8208,100.0
vesseldwt,2,0.0
stevedorenames,2,0.0


In [517]:
def check_duplicates(dataset):
    duplicates = dataset[dataset.duplicated()]
    print(f'There are {len(duplicates)} duplicates.')

In [518]:
check_duplicates(data)

There are 0 duplicates.


We remove the rows with missing values as well as the column with plently of missing data. Furthermore, we remove the date time variables to simplify the problem:

In [519]:
data.drop(['hasnohamis', 'eta','ata','atd', 'earliesteta', 'latesteta'], axis=1, inplace=True)
data.dropna(inplace=True)

We now fix the type of some features: 

In [520]:
var_list = ['vesseltype', 'previousportid', 'nextportid', 'vesselid']
data[var_list] = data[var_list].astype('object') 

In [521]:
data.dtypes

vesseldwt         float64
vesseltype         object
discharge1        float64
load1             float64
discharge2        float64
load2             float64
discharge3        float64
load3             float64
discharge4        float64
load4             float64
stevedorenames     object
traveltype         object
previousportid     object
nextportid         object
isremarkable       object
vesselid           object
dtype: object

Let's look again at the final shape of our dataset:

In [522]:
print(f'The test data has {data.shape[0]} rows and {data.shape[1]} columns')

The test data has 8204 rows and 16 columns


We now create new features containing the total transships:

In [523]:
data['transships1'] = data['discharge1'] + data['load1']
data['transships2'] = data['discharge2'] + data['load2']
data['transships3'] = data['discharge2'] + data['load3']
data['transships4'] = data['discharge4'] + data['load4']
data['ts'] = list(zip(data['transships1'], data['transships2'], data['transships3'], data['transships4']))

In [524]:
data = data[data.columns.drop(list(data.filter(regex='discharge')))]
data = data[data.columns.drop(list(data.filter(regex='load')))]

In [525]:
y= data.pop('ts')
data_train, data_test, y_train, y_test = train_test_split(data, y, train_size=0.8, test_size=0.2, random_state=0)

In [526]:
y_1 = data_train.pop('transships1')
y_2 = data_train.pop('transships2')
y_3 = data_train.pop('transships3')
y_4 = data_train.pop('transships4') 

data_test = data_test[data_test.columns.drop(list(data_test.filter(regex='transships')))]

In [527]:
data_train.dtypes

vesseldwt         float64
vesseltype         object
stevedorenames     object
traveltype         object
previousportid     object
nextportid         object
isremarkable       object
vesselid           object
dtype: object

In [528]:
cat_ft = data_train.select_dtypes('object').columns
num_ft = data_train.select_dtypes('float64').columns

In [529]:
# Apply one-hot encoder to each column with categorical data
encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
train_encoded = pd.DataFrame(encoder.fit_transform(data_train[cat_ft]))
test_encoded = pd.DataFrame(encoder.transform(data_test[cat_ft]))

# Redefine index
train_encoded.index = data_train.index
test_encoded.index = data_test.index

# Set informative feature names
train_encoded.columns = encoder.get_feature_names_out(cat_ft)
test_encoded.columns = encoder.get_feature_names_out(cat_ft)

# Fetch numerical variable
num_train = data_train.drop(cat_ft, axis=1)
num_test = data_test.drop(cat_ft, axis=1)

# Add one-hot encoded categorical data to numerical features
train_encoded = pd.concat([num_train, train_encoded], axis=1)
test_encoded = pd.concat([num_test, test_encoded], axis=1)



In [530]:
scaler = StandardScaler()

train_scaled= pd.DataFrame(scaler.fit_transform(train_encoded[num_ft]))
test_scaled= pd.DataFrame(scaler.transform(test_encoded[num_ft]))

# Redefine index
train_scaled.index = train_encoded.index
test_scaled.index = test_encoded.index

train_scaled.columns = num_ft
test_scaled.columns = num_ft

# Fetch encoded features 
cat_train = train_encoded.drop(num_ft, axis=1)
cat_test = test_encoded.drop(num_ft, axis=1)

# Add scaled numerical variables to encoded features
train_prep = pd.concat([cat_train, train_scaled], axis=1)
test_prep = pd.concat([cat_test, test_scaled], axis=1)

In [531]:
train_prep[num_ft].describe()

,vesseldwt
count,6.563000e+03
mean,-3.247948e-17
std,1.000076e+00
min,-7.194255e-01
25%,-6.034512e-01
50%,-4.794618e-01
75%,1.728163e-01
max,5.494227e+00


In [532]:
reg_model = LinearRegression().fit(train_prep, y_1)

In [533]:
y_pred = reg_model.predict(test_prep)